This notebook will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [21]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

Retrieving data and putting it on a dataframe

In [63]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)\

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

We drop the columns that does not have a Borough and get the coordinates for each postalcode 

In [64]:
df=df[df.Borough != 'Not assigned']

def get_geocode(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We merge both dataframes

In [65]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df, on='Postalcode')

geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


We check for the coordinates for the University of Toronto 

In [66]:
geo_merged.loc[geo_merged['Postalcode']=='M5S',['Postalcode','Latitude','Longitude','Borough']]

,Postalcode,Latitude,Longitude,Borough
66,M5S,43.662696,-79.400049,Downtown Toronto


In [67]:
from math import radians, cos, sin, asin, sqrt 

In [68]:
def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

In [77]:
# define the dataframe columns
column_names = ['Postalcode','Latitude', 'Longitude'] 

# instantiate the dataframe
Testdf.iloc[0:0]
Testdf = pd.DataFrame(columns=column_names)

Testdf.head()

,Postalcode,Latitude,Longitude


In [71]:
dist=distance(43.662696, 43.784535, -79.400049, -79.160497)
print(dist)

23.53952525247996


In [78]:
dist=0
geo_df.reset_index
for index, row in geo_df.iterrows():
    lat=row['Latitude']
    lon=row['Longitude']
    pos=row['Postalcode']
    dist=distance(43.662696, lat, -79.400049, lon)
    if (dist<1.5):
        Testdf = Testdf.append({'Postalcode':pos,'Latitude':lat,'Longitude':lon},ignore_index=True)
Testdf.head()

,Postalcode,Latitude,Longitude
0,M4Y,43.665860,-79.383160
1,M5G,43.657952,-79.387383
2,M5R,43.672710,-79.405678
3,M5S,43.662696,-79.400049
4,M5T,43.653206,-79.400049


In [79]:
Testdf.head(10)

,Postalcode,Latitude,Longitude
0,M4Y,43.665860,-79.383160
1,M5G,43.657952,-79.387383
2,M5R,43.672710,-79.405678
3,M5S,43.662696,-79.400049
4,M5T,43.653206,-79.400049
5,M7A,43.662301,-79.389494


In [80]:
Testdf_merged = pd.merge(Testdf, df, on='Postalcode')

Testdf_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M4Y,43.665860,-79.383160,Downtown Toronto,Church and Wellesley
1,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street
2,M5R,43.672710,-79.405678,Central Toronto,"The Annex, North Midtown, Yorkville"
3,M5S,43.662696,-79.400049,Downtown Toronto,"University of Toronto, Harbord"
4,M5T,43.653206,-79.400049,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"


In [8]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [81]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Testdf_merged['Latitude'], Testdf_merged['Longitude'], Testdf_merged['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [82]:
CLIENT_ID = 'GQYT30GWMF1LZVU5CC4KYN4QDK0444ARNVH5MLEM42FSVBBI' 
CLIENT_SECRET = 'HQ1FSIY0PWJCR41NAHNFCZKT2XSRFC4GY1EQKU55WTTLJVJO'
VERSION = '20180604'

In [83]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=20
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            #Get API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            #Make GET result
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #Return only relevant info
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
toronto_venues = getNearbyVenues(names=Testdf_merged['Neighborhood'],
                                   latitudes=Testdf_merged['Latitude'],
                                   longitudes=Testdf_merged['Longitude']
                                  )

Church and Wellesley
Central Bay Street
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
Queen's Park, Ontario Provincial Government


In [91]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Church and Wellesley,43.66586,-79.38316,DanceLifeX Centre,43.666956,-79.385297,Dance Studio
1,Church and Wellesley,43.66586,-79.38316,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant
2,Church and Wellesley,43.66586,-79.38316,The Alley,43.665922,-79.385567,Bubble Tea Shop
3,Church and Wellesley,43.66586,-79.38316,Smith,43.666927,-79.381421,Breakfast Spot
4,Church and Wellesley,43.66586,-79.38316,Fabarnak,43.666377,-79.380964,Restaurant


In [86]:
Tc=toronto_venues.groupby('Neighborhood').count()
Tc.drop(['Venue', 'Neighborhood Latitude', 'Neighborhood Longitude','Venue Latitude','Venue Longitude'], axis=1,inplace=True)
Tc.rename(columns={"Venue Category": "Venues counted per neighborhood"},inplace=True)
Tc

,Venues counted per neighborhood
Neighborhood,
Central Bay Street,20
Church and Wellesley,20
"Kensington Market, Chinatown, Grange Park",20
"Queen's Park, Ontario Provincial Government",20
"The Annex, North Midtown, Yorkville",20
"University of Toronto, Harbord",20


In [94]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(120, 64)

In [97]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Beer Bar,Belgian Restaurant,Bookstore,Breakfast Spot,...,Sandwich Place,Spa,Sushi Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Central Bay Street,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.05,0.05,0.05,0.00,0.00,0.00,0.0,0.00,0.00,0.00
1,Church and Wellesley,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.05,...,0.00,0.00,0.00,0.00,0.05,0.00,0.0,0.00,0.05,0.00
2,"Kensington Market, Chinatown, Grange Park",0.00,0.05,0.00,0.05,0.05,0.05,0.05,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.1,0.05,0.00,0.00
3,"Queen's Park, Ontario Provincial Government",0.00,0.05,0.00,0.00,0.00,0.05,0.00,0.00,0.00,...,0.00,0.00,0.10,0.05,0.00,0.00,0.0,0.00,0.00,0.05
4,"The Annex, North Midtown, Yorkville",0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,...,0.15,0.00,0.00,0.00,0.00,0.05,0.0,0.00,0.00,0.00


In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Bay Street,Coffee Shop,Japanese Restaurant,Art Museum,Ramen Restaurant,Gastropub,Comic Shop,Italian Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park
1,Church and Wellesley,General Entertainment,Restaurant,Mexican Restaurant,Ice Cream Shop,Burger Joint,Park,Bubble Tea Shop,Pizza Place,Breakfast Spot,Coffee Shop
2,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Organic Grocery,Gourmet Shop,Dessert Shop,Mexican Restaurant,Coffee Shop,Cocktail Bar,Cheese Shop,Caribbean Restaurant
3,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Sushi Restaurant,Park,Italian Restaurant,Burrito Place,Creperie,Hobby Shop,Mexican Restaurant,Yoga Studio
4,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pub,Liquor Store,Indian Restaurant,Middle Eastern Restaurant,Burger Joint,Park,Pharmacy


In [101]:
toronto_grouped[['Neighborhood','Café','Bar','Pub','Park','Pharmacy']]

,Neighborhood,Café,Bar,Pub,Park,Pharmacy
0,Central Bay Street,0.00,0.00,0.00,0.05,0.00
1,Church and Wellesley,0.00,0.00,0.00,0.05,0.00
2,"Kensington Market, Chinatown, Grange Park",0.15,0.05,0.00,0.00,0.00
3,"Queen's Park, Ontario Provincial Government",0.00,0.00,0.00,0.05,0.00
4,"The Annex, North Midtown, Yorkville",0.15,0.00,0.05,0.05,0.05
5,"University of Toronto, Harbord",0.05,0.05,0.00,0.00,0.00
